# **A Baseline Model For Cluster Embeddings**

This notebook contains the baseline model for cluster embeddings. This is a very simple model, using convolutional layers to transform images into an embeddings space of length N. These embeddings are trained by then using them in a classification loss. This will train the model to produce meaningful embeddings, resulting in good classification scores.

## **Notebook Overview**

## **Table of Contents**

1. [Imports & Setup](#Imports-&-Setup)
2. [Model Creation](#Model-Creation)
3. [Dataset Creation](#Dataset-Creation)
4. [Training](#Training)
5. [Clustering Embeddings Analysis](#Clustering-Embeddings-Analysis)
6. [Volume Of Embeddings Space](#Volume-Of-Embeddings-Space)
7. [Collect Cluster Info And Plot Mahalanbois Distance](#Collect-Cluster-Info-And-Plot-Mahalanbois-Distance)
8. [Display The Percent Of Volume That Each Cluster Takes Up](#Display-The-Percent-Of-Volume-That-Each-Cluster-Takes-Up)
9. [Visualize The Angles Between Points And Clusters](#Visualize-The-Angles-Between-Points-And-Clusters)
10. [Visualize The Clustering Embeddings](#Visualize-The-Clustering-Embeddings)

## **Imports & Setup**

In [1]:
# Import the necesary packages
import os
import numpy as np
import pandas as pd

import matplotlib
matplotlib.rcParams.update({'font.size':14})
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.patches import Patch
import seaborn as sns

import tensorflow as tf

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE

from scipy.spatial import ConvexHull
from scipy.spatial.distance import mahalanobis, euclidean

In [2]:
# This is done to limit the memory growth on the GPU
# I was having trouble with my GPU memory being filled so I capped the availible memory
# growth here.

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=16_384)])
    except RuntimeError as e:
        print(e)

### **Configuration Object**

This is the configuration object that holds the main parameters we will be using later in our program. If we run our code with Weights and Biases (WANDB) then it will be saved to the given run, allowing us to see the effect of different hyper-parameters.

In [3]:
config = {
    # Basic information
    "AUTHOR": "Kiernan",
    
    # Training params
    "LR": 0.001,
    "BATCH_SIZE": 32,
    "EPOCHS": 50,
    
    # Model params
    "CONV_LAYERS": 8,
    "FIRST_FILTERS": 32,
    "FIRST_KERNEL_SIZE": (5,5),
    "N_FILTERS": 16,
    "KERNEL_SIZE": (3,3),
#     "DENSE_SIZE": 16,
    "DROPOUT": 0.2,
    "EMBEDDING_SIZE": 3,
}

### **Initialize WANDB**

Weights and Biases is a very useful machine learning pipelining tool. It can be super handy when it comes to tracking experiments. It is capable of tracking the loss over epochs, the best model, and the parameters used when training different models. All this information can then be used ot optimize the hyper parameters of a model.

In [4]:
RUN_WANDB = False
if(RUN_WANDB):
    import wandb
    from wandb.keras import WandbCallback
    from secrets import WANDB
    wandb.login(key=WANDB)
    run = wandb.init(project="deep-clustering-evaluation", entity="kmcguigan", group="base-model", config=config, job_type="train")

### **Loading Data**

We created the data ahead of time to make these experiments more reproducible.

In [5]:
with open('data/train.npy', mode='rb') as infile:
    X_train = np.load(infile, allow_pickle=True)
    y_train = np.load(infile, allow_pickle=True)

with open('data/val.npy', mode='rb') as infile:
    X_val = np.load(infile, allow_pickle=True)
    y_val = np.load(infile, allow_pickle=True)

with open('data/test.npy', mode='rb') as infile:
    X_test = np.load(infile, allow_pickle=True)
    y_test = np.load(infile, allow_pickle=True)
    
with open('data/viz.npy', mode='rb') as infile:
    X_viz = np.load(infile, allow_pickle=True)
    y_viz = np.load(infile, allow_pickle=True)

print(f"Train data shape: {X_train.shape} Val data shape: {X_val.shape} Test data shape: {X_test.shape}")

Train data shape: (50000, 28, 28, 1) Val data shape: (10000, 28, 28, 1) Test data shape: (10000, 28, 28, 1)


### **Load in the Labels**

This dataframe maps the labels to their corresponding names. For classification we want numeric labels going from 1...N where N is the number of classes we have. In certain scenarios (like on MNIST) the label matches the name, but for other datasets we would want to map the numeric label to the human understandable name we associate with each class.

In [6]:
labels_df = pd.read_csv('data/label_names.csv')
labels_df['label'] = labels_df['label'].astype(int)
labels_df

,label,name
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
5,5,5
6,6,6
7,7,7
8,8,8
9,9,9


## **Model Creation**

### **Create Model Body**

The body of the model is a very simple convolutional architecture. After a number of convolutional layers we finish with global average pooling which takes the average of all the pixels in each channel, returning a vector of (Batch Size, Number of Chanels). This vector can now be considered the embedding vector, representing the components of the image.

In [7]:
def create_body(image_shape):
    inputs = tf.keras.layers.Input(shape=image_shape)
    
    def cov_block(layer_inputs, n_filters, kernel_size, strides=1):
        x = tf.keras.layers.Conv2D(n_filters, kernel_size, padding="same", strides=strides)(layer_inputs)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
        return x
    
    x = cov_block(inputs, config["FIRST_FILTERS"], config["FIRST_KERNEL_SIZE"])
    for _ in range(config["CONV_LAYERS"]):
        x = cov_block(x, config["N_FILTERS"], config["KERNEL_SIZE"])
        
    x = tf.keras.layers.Conv2D(config["EMBEDDING_SIZE"], config["KERNEL_SIZE"], padding="same")(x)
    outputs = tf.keras.layers.GlobalAveragePooling2D()(x)
    return tf.keras.models.Model(inputs=inputs, outputs=outputs, name="body")

body = create_body(X_train.shape[1:])
body.summary()

Metal device set to: Apple M1 Pro


2022-12-09 16:17:37.433045: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-09 16:17:37.433615: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "body"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 28, 28, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 16)        4624      
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 16)       64        
 hNormalization)                                              

### **Create the Model Head**

The head is what makes the actual predictions. A good way to think about this is that the body learns to extract the features from the images, then the head uses these extracted features to classify which number is represented.

In [8]:
def create_head(n_classes):
    inputs = tf.keras.layers.Input(shape=(config["EMBEDDING_SIZE"]))
#     x = tf.keras.layers.Dropout(config["DROPOUT"])(inputs)
    outputs = tf.keras.layers.Dense(n_classes, activation='softmax')(inputs)
    return tf.keras.models.Model(inputs=inputs, outputs=outputs, name="head")

head = create_head(10)
head.summary()

Model: "head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 10)                40        
                                                                 
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________


### **Create the Overall Model**

Here we combine the body and the head to a full on model, we attatch the loss function and this gives us our trainable model.

In [9]:
model = tf.keras.models.Sequential([
    body,
    head
],
    name="combinedModel"
)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=config['LR'])
metrics = [tf.keras.metrics.SparseCategoricalAccuracy(),]
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
model.summary()

Model: "combinedModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 body (Functional)           (None, 3)                 22771     
                                                                 
 head (Functional)           (None, 10)                40        
                                                                 
Total params: 22,811
Trainable params: 22,491
Non-trainable params: 320
_________________________________________________________________


## **Training**

Here we train the model, this uses both early stopping and a learning rate reducer to prevent overfitting and help us get the best possible model.

In [10]:
stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
if(RUN_WANDB):
    callbacks = [stopper, lr_reducer, WandbCallback(predictions=8, input_type='images', validation_data=(X_val, y_val))]
else:
    callbacks = [stopper, lr_reducer]

hist = model.fit(X_train, y_train,
                 validation_data=(X_val, y_val),
                 batch_size=config["BATCH_SIZE"],
                 validation_batch_size=config["BATCH_SIZE"],
                 epochs=config["EPOCHS"],
                 shuffle=True,
                 callbacks=callbacks)

Epoch 1/50


2022-12-09 16:17:37.985432: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-09 16:17:38.608650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - ETA: 0s - loss: 0.5991 - sparse_categorical_accuracy: 0.8034

2022-12-09 16:18:09.493909: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1563/1563 [==============================] - 34s 21ms/step - loss: 0.5991 - sparse_categorical_accuracy: 0.8034 - val_loss: 0.5919 - val_sparse_categorical_accuracy: 0.8174 - lr: 0.0010
Epoch 2/50
 307/1563 [====>.........................] - ETA: 23s - loss: 0.2520 - sparse_categorical_accuracy: 0.9255


KeyboardInterrupt



### **Vizualize Training Results**

This shows how well the model learned what it intended to. We can see how the leraning rate was reduced, the accuracy of predictions, and the loss value.

In [ ]:
fig = plt.figure(figsize=(12,12), tight_layout=True)

plt.subplot(311)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')

plt.subplot(312)
plt.plot(hist.history['sparse_categorical_accuracy'])
plt.plot(hist.history['val_sparse_categorical_accuracy'])
plt.title('Model Categorical Accuracy')
plt.ylabel('Categorical Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')

plt.subplot(313)
plt.plot(hist.history['lr'])
plt.title('Learining Rate')
plt.ylabel('Learning Rate')
plt.xlabel('Epoch')

if(RUN_WANDB):
    run.log({'train_graph': wandb.Image(fig, caption="Training Graph")})
plt.show()

### **Analyze Results on the Test Set**

In [ ]:
ev = model.evaluate(X_test, y_test)
if(RUN_WANDB):
    run.log({'test/loss':ev[0], 'test/categorical_accuracy':ev[1]})
ev

## **Clustering Embeddings Analysis**

Here we will use kmeans to create clusters of the embeddings. Since we know the number of labels we will use this as our K-value (this is the number of clusters). We then go through each cluster and find the dominant label in this cluster. We make the assumption here that the network performed half decent and that most samples are correctly labelled. For the purposes of this project, which is to show off the effectiveness of different clustering techniques, this works fine. For different projects it would be advisable to not make this assumption, maybe classifying samples with a K-Nearest Neighbors classifier based on the labelled training data.

Cluster accuracy represents the fraction of samples that are in the correct cluster, or in other words the number of samples that are found in the cluster with a matching dominant label. We can then create a confusion matrix showing the actual and predicted class probabilities.

In [ ]:
def kmeans_cluster_accuracy(embeddings, y):
    # run k-means
    all_labels = labels_df['label'].values
    kmeans = KMeans(n_clusters=len(all_labels), random_state=123)
    labels = kmeans.fit_predict(embeddings)
    
    # find the mapping of each kmeans label to the actual label
    label_mappings = {}
    for label in all_labels:
        values, counts = np.unique(y[np.where(labels==label)], return_counts=True)
        label_mappings[label] = values[np.argmax(counts)]
    
    # map kmeans labels to actual label
    map_labels = np.vectorize(lambda x: label_mappings[x])
    mapped_labels = map_labels(labels)
    
    # calculate the centers for each mapped labels
    kmeans_clusters = kmeans.cluster_centers_
    mapped_clusters = {}
    for label in all_labels:
        mapped_clusters[label_mappings[label]] = kmeans_clusters[label,:]
    return accuracy_score(y.reshape((-1,1)), mapped_labels.reshape((-1,1))), mapped_labels, mapped_clusters

embeddings = body.predict(X_test)
accuracy, preds, cluster_centers = kmeans_cluster_accuracy(embeddings, y_test)
# run.log({'test/clustering_accuracy':accuracy})
print(f"Cluster Accurancy {accuracy}")

In [ ]:
cf = confusion_matrix(y_test, preds, normalize='true')
class_names = labels_df['name'].values
fig, ax = plt.subplots(1,1,figsize=(24,16))
sns.heatmap(cf, annot=True, xticklabels=class_names, yticklabels=class_names, cmap='Blues', robust=True, ax=ax, annot_kws={"size": 14})
ax.set_ylabel("Actual Label", fontdict={'fontsize':16})
ax.set_xlabel("Predicted Label", fontdict={'fontsize':16})
if(RUN_WANDB):
    run.log({'confusion_matrix': wandb.Image(fig, caption="Confusion Matrix")})
plt.show()

In [ ]:
if(RUN_WANDB):
    run.log({
        "testExamples": [wandb.Image(im, caption=f"Pred:{pred} Lable:{y}")
                            for im, pred, y in zip(X_test[:16,:,:,:], np.argmax(preds[:16], axis=-1), y_test[:16])]
    })

## **Volume Of Embeddings Space**

First we will get the volume of the overall embeddings space. This will help us get a baseline for the total space that was learned here. This will make it easier to compare embeddings later on.

In [ ]:
from scipy.spatial import ConvexHull
hull = ConvexHull(embeddings)
overall_volume = hull.volume
overall_volume

## **Collect Cluster Info And Plot Mahalanbois Distance**

Here we calculated the covariance matrix for the different clusters. This will let us calculate the Mahalanbois distance for the positive and negative points in the cluster. Since our embeddings space is not 1-dimensional our clusters are not 1-dimensional either. Since our clusters may vary differently on the different basis vectors a simple euclidean distance might not make sense. Picture an oval with semi-axis 10 on the x-axis and 5 on the y-axis. A point 10 units away on the x-axis is closer to the mass of the oval than a point 10 units away on the y-axis. This is where Mahalanbois distance comes in. This gives us the covariance adjusted distance from the cluster. Letting us easily compare the true positives and false positives classification within a cluster.


#### **Calculating the Volume of a Cluster**
The next thing we do is we also calculate the volume of the embeddings for the cluster. This is done in two ways. The first is simple and we get the volume of all points in the cluster. This is fine but a thing to note is that there will always be outlier points, and maybe we don't want to worry about those few outliers so much. Well we have a covariance matrix which tells us the covariance and the correlation between covariance in the different dimensions. 

We will start by thinking of the covariance matrix as some linear transformation from standard basis vecotrs. Now unless the correlation between the different dimensions is independent, meaning we have zeros for any cells off the diagonal of the covariance matrix, this linear transformation is throwing the basis vectors off their span making them uselss to calculate the volume of the ellipsoid. You can already probably guess that this is where the eigen vectors and eigen values come in. The eigenvectors will be the new basis vectors we consider as they aren't thrown off their span, and the eigen-values will be the scalling factor each of these basis vectors undergo. Since we want volume, and don't care so much about orientation we can just pretend we have an axis aligned error ellipsoid with the semi-axis being the square root (since variance is the squared standard deviation) of the eigen-values.

With the standard deviation in each direction, and new basis vectors that make the dimensions of our error ellisoid independent from one another, we can write the error ellipsoid like we would any standard axis-aligned ellipsoid like (x / sqrt(eigen-value-1))^2 + (y / sqrt(eigen-value-2))^2 + ... = S where is some scalling factor. This is handy becuase based on the covariance matrix we know the values of say x are normally distributed, we also subtracted the centroid when calculating this so the mean of this distribution is zero. Since we are dividing by the sqrt of the eigen value, this is the same thing as dividing by the standard deviation, which gives us a standard normal random vairable. 

Becuase we take the square of this standard normal random variable and add them together this is the difinition of a chi-squared distribution. Since the degrees of freedom is just the number of dimensions, we can use the lookup table to find the chi-squared critical value for n degrees of freedom and a 95% confidence level. 

A question I had myself was why this equation works for different sizes of error ellipsoids, the reason is that we have standardized normal random variables, we are accounting for the size of the independent random normal variables. This is why the scalling factor from the chi-squared distribution is the same for a given degrees of freedom and confidence level. I find it cool how all these things relate together.

At this point you may or may not be reading this explanation, I don't blame you either way. The fact you're reading this part means you're still with me so here we go. We have this cool formula but now how do we get the semi-axis lengths? Well this is the easy part, essentially we want the semi-axis lengths when all other semi-axis are zero. In other words the axis aligned ellipse semi-axis length on the x dimension is the length when the length on the y dimension is 0. Essentially the formula looks like this semi-axis-length = sqrt(S) * sqrt(eigen-value). Pretty simple right?

https://www.visiondummy.com/2014/04/draw-error-ellipse-representing-covariance-matrix/

https://cookierobotics.com/007/

In [ ]:
PLOT = True

def get_conf_interval(distances, indecies=None):
    if(indecies is not None):
        distances = distances[indecies]
    conf_interval = [np.min(distances),]
    for inter in [2,5,25,75,95,98]:
        conf_interval.append(np.percentile(distances,inter))
    conf_interval.append(np.max(distances))
    return conf_interval

def cosine_similarity(samples, point):
    samples = samples / np.linalg.norm(samples, axis=-1).reshape(-1,1)
    point = point / np.linalg.norm(point)
    cosine = samples @ point
    cosine = np.clip(cosine, -1.0, 1.0)
    theta = np.arccos(cosine)
    return theta
    
cluster_info = {}
for cluster in labels_df['label'].values:
    # pull the data we work with for this cluster
    indecies = np.where(preds==cluster)[0]
    samples = embeddings[indecies]
    actuals = y_test[indecies]
    centroid = cluster_centers[cluster]

    # grab the indecies of both positive and negative elements in the cluster
    positive_indecies = np.where(actuals==cluster)[0]
    negative_indecies = np.where(actuals!=cluster)[0]
    
    # calculate the covariance matrix of all samples
    cov = np.cov((samples - centroid).T)
    inv_cov = np.linalg.inv(cov)
    
    # calculate sample distances
    sample_mahalanbois_distances = np.array([mahalanobis(centroid, samples[idx,:], inv_cov) for idx in range(samples.shape[0])])
    sample_theta_distances = cosine_similarity(samples, centroid)
    
    # calculate the confidence interval for point densities
    conf_interval_mahalanobis = get_conf_interval(sample_mahalanbois_distances)
    conf_interval_theta = get_conf_interval(sample_theta_distances)
    
    # calculate the volume of the covariance matrix
    hull = ConvexHull(samples)
    volume = hull.volume
    
    eigen_values, eigen_vectors = np.linalg.eig(cov)
    if(config["EMBEDDING_SIZE"]==3):
        get_side_length = np.vectorize(lambda x: np.sqrt(7.815*x))
        semi_axes_lengths = get_side_length(eigen_values)
        confidence_volume = (4 / 3) * np.pi * semi_axes_lengths[0] * semi_axes_lengths[1] * semi_axes_lengths[2]
    elif(config["EMBEDDING_SIZE"]==2):
        get_side_length = np.vectorize(lambda x: np.sqrt(5.991*x))
        semi_axes_lengths = get_side_length(eigen_values)
        confidence_volume = np.pi * semi_axes_lengths[0] * semi_axes_lengths[1]
    else: 
        raise Exception(f'Invalid dimensions {len(semi_axes_lengths)}')
    
    # record the cluster information
    cluster_info[cluster] = {
        'cov':cov,
        'inv_cov':inv_cov,
        'conf_interval_mahalanobis':conf_interval_mahalanobis,
        'conf_interval_theta':conf_interval_theta,
        'positive_examples':len(positive_indecies),
        'negative_examples':len(negative_indecies),
        'volume':volume,
        'confidence_volume':confidence_volume,
    }
    
    if(PLOT):
        # plot the distances
        fig, ax = plt.subplots(1,1,figsize=(22,12))
        ax.set_title(f'Cluster {cluster} Mahalanobis Distances', fontsize=24)
        # create the dataframe to show or properly label the plots
        g1 = sns.histplot(data=sample_mahalanbois_distances[positive_indecies], stat='percent', color='mediumblue', kde=True, ax=ax, bins=20)
        g2 = sns.histplot(data=sample_mahalanbois_distances[negative_indecies], stat='percent', color='orange', kde=True, ax=ax, bins=20)
        # update the labels for the legend
        legend_elements = [
            Patch(facecolor='mediumblue',edgecolor='blue',label='Correctly Labelled Points'),
            Patch(facecolor='orange',edgecolor='red',label='Incorrectly Labelled Points'),
        ]
        plt.legend(handles=legend_elements, loc='best', fontsize=20)
        ax.set_xlim(left=-0.001)
        plt.show()
        print("")

## **Collect Cluster-to-Cluster Analysis Information**

Here we want to get some information about the other clusters. There are a lot of statistics that are calculated and not used such as all the confidence intervals on distances. I left the code here as some ideas for more ways to dive into the results. 

In [ ]:
cluster_to_cluster = {}
for cluster in labels_df['label'].values:
    # pull the data we work with for this cluster
    indecies = np.where(preds==cluster)[0]
    samples = embeddings[indecies]
    actuals = y_test[indecies]
    centroid = cluster_centers[cluster]
    
    # iterate over other clusters to see how these points stack up on them
    for compare_cluster in labels_df['label'].values:
        if(cluster == compare_cluster):
            continue
            
        # get the cluster we will compare against
        comparative_centroid = cluster_centers[compare_cluster]
        
        # get the distance between the comparative centroid and this one
        centroid_distance_mahalanbois = mahalanobis(centroid, comparative_centroid, cluster_info[cluster]['inv_cov'])
        centroid_distance_theta = cosine_similarity(centroid.reshape(1,-1), comparative_centroid)
        
        # get the distance between points in this cluster
        sample_mahalanbois_distances = np.array([mahalanobis(comparative_centroid, samples[idx,:], cluster_info[compare_cluster]['inv_cov']) for idx in range(samples.shape[0])])
        sample_theta_distances = cosine_similarity(samples, comparative_centroid)
    
        # grab the indecies of both positive and negative elements in the cluster
        positive_indecies = np.where(actuals==compare_cluster)[0]
        negative_indecies = np.where(actuals!=compare_cluster)[0]
        
        # calculate the confidence interval for point densities
        conf_interval_mahalanobis = get_conf_interval(sample_mahalanbois_distances)
        conf_interval_theta = get_conf_interval(sample_theta_distances)
        
        # record what we learned
        d = cluster_to_cluster.get(compare_cluster, dict())
        d[cluster] = {
            'centroid_distance_mahalanbois':centroid_distance_mahalanbois,
            'centroid_distance_theta':centroid_distance_theta,
            'conf_interval_mahalanobis':conf_interval_mahalanobis,
            'conf_interval_theta':conf_interval_theta,
            'positive_samples':len(positive_indecies),
            'negative_samples':len(negative_indecies)
        }
        cluster_to_cluster[compare_cluster] = d

## **Display The Percent Of Volume That Each Cluster Takes Up**

Here we display the overall and 95% confidence level volumes that the clusters take up. This is a good way of showing how tight the clusters are compared to the space aloted. This is not a perfect measure but we can look at the differences between the overall and confidence level volumes. We might also be interested in the percent of the total volume that is covered by points. The tigher the clusters the lower the volume percent. This will be skewed if the overall volume is smaller, but using the 95% to total comparison we can get an idea for the ratio.

In [ ]:
volumes = []
confidence_volumes = []
for cluster in cluster_info.keys():
    volume_ratio = (cluster_info[cluster]['volume'] / overall_volume) * 100
    confidence_volume_ratio = (cluster_info[cluster]['confidence_volume'] / overall_volume) * 100
    volumes.append(volume_ratio)
    confidence_volumes.append(confidence_volume_ratio)
    print(f"Cluster {cluster} => Volume Ratio: {volume_ratio:,.3f}% 95% Confidence Volume Ratio: {confidence_volume_ratio:,.3f}%")
    
print("")
print("Overall Statistics")
print(f"Mean Values   => Mean Volume Ratio: {np.mean(volumes):,.3f}% Mean 95% Confidence Volume Ratio: {np.mean(confidence_volumes):,.3f}%")
print(f"Summed Values => Total Volume Ratio: {np.sum(volumes):,.3f}% Total 95% Confidence Volume Ratio: {np.sum(confidence_volumes):,.3f}%")
print("")
print(f"Total Volume: {overall_volume:,.0f}")

## **Visualize The Angles Between Points And Clusters**

Here is the important visualization. For each cluster we find the nearest cluster (given the angular distance between centroids) and we plot the angle between the points of the given cluster to the nearest cluster and vise-versa. This is one that is size agnostic, we can see the tighness of the clusters. The more centered the clusters the better the embeddings are. 

In [ ]:
for cluster in cluster_to_cluster.keys():
    # get the samples
    indecies = np.where(preds==cluster)[0]
    samples = embeddings[indecies]
    # find the closest cluster to this one
    closest_cluster, distance = None, None
    for other_cluster in cluster_to_cluster[cluster].keys():
        if(distance is None or cluster_to_cluster[cluster][other_cluster]['centroid_distance_theta']<distance):
            closest_cluster = other_cluster
            distance = cluster_to_cluster[cluster][other_cluster]['centroid_distance_theta']
    # get the samples for the closest cluster
    closest_indecies = np.where(preds==closest_cluster)[0]
    closest_samples = embeddings[closest_indecies]
    # get the angular distances
    this_to_this_theta = cosine_similarity(samples, cluster_centers[cluster])
    this_to_other_theta = cosine_similarity(samples, cluster_centers[closest_cluster])
    other_to_this_theta = cosine_similarity(closest_samples, cluster_centers[cluster])
    other_to_other_theta = cosine_similarity(closest_samples, cluster_centers[closest_cluster])
    # plot the angular distance between points
    _, ax = plt.subplots(1,1,figsize=(22,8))
    ax.scatter(this_to_this_theta, this_to_other_theta, color='blue', label=f'Samples from Cluster {cluster}', alpha=0.5)
    ax.scatter(other_to_this_theta, other_to_other_theta, color='red', label=f'Samples from Cluster {closest_cluster}', alpha=0.5)
    ax.set_title(f'Comparison of Angles Between Cluster Points of Cluster {cluster} & Cluster {closest_cluster}', fontdict={'fontsize':24})
    ax.set_xlabel(f'Theta Between Point and {cluster} Cluster', fontsize=18)
    ax.set_ylabel(f'Theta Between Point and {closest_cluster} Cluster', fontsize=18)
    ax.legend(loc="best")
    plt.show()
    print("")

## **Visualize The Clustering Embeddings**

Here we try and see what the clustering looks like. I visualize this in both 2D and 3D. The reason for this is that TSNE is good at keeping items that are close in a higher dimension close in a lower one, but it isint always good when it comes to shapes and when comparing relative distances. What I mean by relative distances is say there is a point x on the edge of cluster A, the distance A -> x in TSNE cannot be directly compared to say the distance of point y on edge of cluster B since these distances can be manipulated in the transform.

A detailed TSNE Overview

https://distill.pub/2016/misread-tsne/

I found the code to make the very nice visualizations here

https://towardsdatascience.com/visualizing-feature-vectors-embeddings-using-pca-and-t-sne-ef157cea3a42#:~:text=t%2Ddistributed%20stochastic%20neighbour%20embedding,non%2Dlinear%20dimensionality%20reduction%20technique.


In [ ]:
def visualize_embeddings_2d(embeddings, y):
    if(embeddings.shape[1] != 2):
        tsne = TSNE(n_components=2, learning_rate='auto', init='random', verbose=1, perplexity=50)
        embeddings = tsne.fit_transform(embeddings)
    cmap = cm.get_cmap('tab20')
    fig, ax = plt.subplots(figsize=(22,12))
    num_categories = 10
    for lab in range(num_categories):
        indices = y==lab
        ax.scatter(
            embeddings[indices,0],
            embeddings[indices,1], 
            c=np.array(cmap(lab)).reshape(1,4), 
            label=labels_df['name'].values[lab], 
            alpha=0.5
        )
    ax.legend(fontsize='large', markerscale=2)
    if(RUN_WANDB):
        run.log({'tsne_embeddings': wandb.Image(fig, caption="Test Embeddings")})
    plt.show()
    
def visualize_embeddings_3d(embeddings, y):
    if(embeddings.shape[1] != 3):
        tsne = TSNE(n_components=3, learning_rate='auto', init='random', verbose=1)
        embeddings = tsne.fit_transform(embeddings)
    cmap = cm.get_cmap('tab20')
    fig = plt.figure(figsize=(22,12))
    ax = fig.add_subplot(projection='3d')
    num_categories = 10
    for lab in range(num_categories):
        indices = y == lab
        ax.scatter(embeddings[indices, 0],
                   embeddings[indices, 1],
                   embeddings[indices, 2],
                   c=np.array(cmap(lab)).reshape(1, 4),
                   label=lab,
                   alpha=0.5)
    ax.legend(fontsize='large', markerscale=2)
    if(RUN_WANDB):
        run.log({'tsne3d_embeddings': wandb.Image(fig, caption="Test Embeddings 3D")})
    plt.show()

embeddings = body.predict(X_test)

In [ ]:
%matplotlib inline
visualize_embeddings_2d(embeddings, y_test)

In [ ]:
%matplotlib inline
visualize_embeddings_3d(embeddings, y_test)

## **Finish the Run and Upload the Results**

In [ ]:
if(RUN_WANDB):
    run.finish()

### **Extra Code Used to Calculate Extra Metrics**

This code is not used in this analysis, but I kept it here to calculate things like the average euclidean distance between points or the average angular distance between points as a keras callback for during model training.

In [ ]:
# def angular_distances(embeddings):
#     embeddings = tf.math.l2_normalize(embeddings, axis=-1)
#     cos = tf.matmul(embeddings, tf.transpose(embeddings))
#     cos = K.clip(cos, -1.0, 1.0)
#     theta = tf.acos(cos)
#     theta = tf.linalg.set_diag(theta, tf.zeros(tf.shape(embeddings)[0]))
#     return theta
    
# def euclidean_distances(embeddings):
#     samples, dims = tf.shape(embeddings)
#     euclidean_distances = []
#     for sample_idx in range(samples):
#         euclidean_distances.append(tf.linalg.norm(embeddings - embeddings[sample_idx,:], axis=-1))
#     euclidean_distances = tf.stack(euclidean_distances)
#     euclidean_distances = tf.linalg.set_diag(euclidean_distances, tf.zeros(tf.shape(euclidean_distances)[0]))
#     return euclidean_distances

# def apply_metric(embeddings, labels, metric):
#     # reshape the labels and find where they match versus the transposed version
#     labels = tf.reshape(labels, (-1,1))
#     adj = tf.equal(labels, tf.transpose(labels))
#     adj_not = tf.math.logical_not(adj)
#     # remove self from the adj list
#     diag = tf.zeros([tf.shape(labels)[0]])
#     diag = tf.cast(diag, tf.bool)
#     adj = tf.linalg.set_diag(adj, diag)
#     # caluclate the distances between points
#     distances = metric(embeddings)
#     pos_dist_mean = tf.reduce_mean(tf.ragged.boolean_mask(distances, mask=adj))
#     neg_dist_mean = tf.reduce_mean(tf.ragged.boolean_mask(distances, mask=adj_not))
#     # calculate the distances between points for each label
#     label_metrics = {}
#     for label in labels_df['label'].values:
#         # get the indecies for this label
#         indecies = np.where(labels==label)[0]
#         # get the label mask
#         label_mask = tf.equal(labels, label)
#         label_mask = tf.repeat(label_mask, tf.shape(label_mask)[0], axis=-1)
#         # get the label distance means
#         label_pos_dist_mean = tf.reduce_mean(tf.ragged.boolean_mask(distances, mask=tf.math.logical_and(adj, label_mask)))
#         label_neg_dist_mean = tf.reduce_mean(tf.ragged.boolean_mask(distances, mask=tf.math.logical_and(adj_not, label_mask)))
#         # record the label distance means
#         label_metrics[label] = (label_pos_dist_mean, label_neg_dist_mean)
#     return pos_dist_mean, neg_dist_mean, label_metrics

In [ ]:
# pos_dist_mean, neg_dist_mean, label_metrics = apply_metric(embeddings, y_test, angular_distances)
# print(f"Positive Angular Distances: {pos_dist_mean}")
# print(f"Negative Angular Distances: {neg_dist_mean}")
# if(RUN_WANDB):
#     run.log({'test/positive_angular_distances':pos_dist_mean,"test/negative_angular_distances":neg_dist_mean})

In [ ]:
# for label, (pos_dist_mean, neg_dist_mean) in label_metrics.items():
#     print(f"{label} Positive Angular Distances: {pos_dist_mean}")
#     print(f"{label} Negative Angular Distances: {neg_dist_mean}")
#     print("")
#     if(RUN_WANDB):
#         run.log({f'test/{label}_positive_angular_distances':pos_dist_mean,f"test/{label}_negative_angular_distances":neg_dist_mean})

In [ ]:
# pos_dist_mean, neg_dist_mean, label_metrics = apply_metric(embeddings, y_test, euclidean_distances)
# print(f"Positive Euclidean Distances: {pos_dist_mean}")
# print(f"Negative Euclidean Distances: {neg_dist_mean}")
# if(RUN_WANDB):
#     run.log({'test/positive_euclidean_distances':pos_dist_mean,"test/negative_euclidean_distances":neg_dist_mean})

In [ ]:
# for label, (pos_dist_mean, neg_dist_mean) in label_metrics.items():
#     print(f"{label} Positive Euclidean Distances: {pos_dist_mean}")
#     print(f"{label} Negative Euclidean Distances: {neg_dist_mean}")
#     print("")
#     if(RUN_WANDB):
#         run.log({f'test/{label}_positive_euclidean_distances':pos_dist_mean,f"test/{label}_negative_euclidean_distances":neg_dist_mean})

In [ ]:
# def get_conf_interval(distances, indecies=None):
#     if(indecies is not None):
#         distances = distances[indecies]
#     conf_interval = [np.min(distances),]
#     for inter in [2,5,25,75,95,98]:
#         conf_interval.append(np.percentile(distances,inter))
#     conf_interval.append(np.max(distances))
#     return conf_interval

# cluster_info = {}
# for cluster in labels_df['label'].values:
#     # pull the data we work with for this cluster
#     indecies = np.where(preds==cluster)[0]
#     samples = embeddings[indecies]
#     actuals = y_test[indecies]
#     centroid = cluster_centers[cluster]

#     # grab the indecies of both positive and negative elements in the cluster
#     positive_indecies = np.where(actuals==cluster)[0]
#     negative_indecies = np.where(actuals!=cluster)[0]
    
#     # calculate the covariance matrix of all samples
# #     cov = np.cov((samples - centroid).T)
#     cov = np.cov(samples.T)
#     inv_cov = np.linalg.inv(cov)
    
#     # calculate sample distances
# #     sample_distances = np.array([euclidean(centroid, samples[idx,:]) for idx in range(samples.shape[0])])
#     sample_mahalanbois_distances = np.array([mahalanobis(centroid, samples[idx,:], inv_cov) for idx in range(samples.shape[0])])
    
# #     # calculate the confidence interval for point densities
#     conf_interval_mahalanobis = get_conf_interval(sample_mahalanbois_distances)
# #     negative_conf_interval_mahalanobis = get_conf_interval(sample_mahalanbois_distances, indecies=negative_indecies)
# #     positive_conf_interval_mahalanobis = get_conf_interval(sample_mahalanbois_distances, indecies=positive_indecies)
    
# #     conf_interval = get_conf_interval(sample_distances)
# #     negative_conf_interval = get_conf_interval(sample_distances, indecies=negative_indecies)
# #     positive_conf_interval = get_conf_interval(sample_distances, indecies=positive_indecies)
    
#     # record the cluster information
#     cluster_info[cluster] = {
#         'cov':cov,
#         'inv_cov':inv_cov,
#         'conf_interval_mahalanobis':conf_interval_mahalanobis,
# #         'positive_conf_interval_mahalanobis':positive_conf_interval_mahalanobis,
# #         'negative_conf_interval_mahalanobis':negative_conf_interval_mahalanobis,
# #         'conf_interval':conf_interval,
# #         'positive_conf_interval':positive_conf_interval,
# #         'negative_conf_interval':negative_conf_interval,
#         'positive_examples':len(positive_indecies),
#         'negative_examples':len(negative_indecies)
#     }
    
#     # plot the distances
#     fig, ax = plt.subplots(1,1,figsize=(22,12))
#     ax.set_title(f'Cluster {cluster} Mahalanobis Distances', fontsize=24)
#     # create the dataframe to show or properly label the plots
#     g1 = sns.histplot(data=sample_mahalanbois_distances[positive_indecies], stat='percent', color='mediumblue', kde=True, ax=ax, bins=20)
#     g2 = sns.histplot(data=sample_mahalanbois_distances[negative_indecies], stat='percent', color='orange', kde=True, ax=ax, bins=20)
#     # update the labels for the legend
#     legend_elements = [
#         Patch(facecolor='mediumblue',edgecolor='blue',label='Correctly Labelled Points'),
#         Patch(facecolor='orange',edgecolor='red',label='Incorrectly Labelled Points'),
#     ]
#     plt.legend(handles=legend_elements, loc='best', fontsize=20)
#     plt.show()
#     print("")


# def positive_cluster_analysis(distances, indecies):
#     if(len(indecies)==0):
#         return (np.nan,np.nan,np.nan)
#     return (
#         np.min(distances[indecies]),
#         np.mean(distances[indecies]),
#         np.max(distances[indecies]),
#     )

# cluster_to_cluster = {}
# for cluster in labels_df['label'].values:
#     # pull the data we work with for this cluster
#     indecies = np.where(preds==cluster)[0]
#     samples = embeddings[indecies]
#     actuals = y_test[indecies]
#     centroid = cluster_centers[cluster]
#     # iterate over other clusters to see how these points stack up on them
#     for compare_cluster in labels_df['label'].values:
#         if(cluster == compare_cluster):
#             continue
            
#         # get the cluster we will compare against
#         comparative_centroid = cluster_centers[compare_cluster]
        
#         # get the distance between the comparative centroid and this one
#         centroid_distance_mahalanbois = mahalanobis(comparative_centroid, centroid, cluster_info[compare_cluster]['inv_cov'])
        
#         # get the distance between points in this cluster
# #         sample_distances = np.array([euclidean(comparative_centroid, samples[idx,:]) for idx in range(samples.shape[0])])
#         sample_mahalanbois_distances = np.array([mahalanobis(comparative_centroid, samples[idx,:], cluster_info[compare_cluster]['inv_cov']) for idx in range(samples.shape[0])])
    
#         # grab the indecies of both positive and negative elements in the cluster
#         positive_indecies = np.where(actuals==compare_cluster)[0]
#         negative_indecies = np.where(actuals!=compare_cluster)[0]
        
#         # calculate the confidence interval for point densities
#         conf_interval_mahalanobis = get_conf_interval(sample_mahalanbois_distances)
# #         negative_conf_interval_mahalanobis = get_conf_interval(sample_mahalanbois_distances, indecies=negative_indecies)
# #         positive_conf_interval_mahalanobis = positive_cluster_analysis(sample_mahalanbois_distances, positive_indecies)
        
# #         conf_interval = get_conf_interval(sample_distances)
# #         negative_conf_interval = get_conf_interval(sample_distances, indecies=negative_indecies)
# #         positive_conf_interval = positive_cluster_analysis(sample_distances, positive_indecies)
        
#         # record what we learned
#         d = cluster_to_cluster.get(compare_cluster, dict())
#         d[cluster] = {
# #             'centroid_distance':centroid_distance,
#             'centroid_distance_mahalanbois':centroid_distance_mahalanbois,
#             'conf_interval_mahalanobis':conf_interval_mahalanobis,
# #             'negative_conf_interval_mahalanobis':negative_conf_interval_mahalanobis,
# #             'positive_conf_interval_mahalanobis':positive_conf_interval_mahalanobis,
# #             'conf_interval':conf_interval,
# #             'negative_conf_interval':negative_conf_interval,
# #             'positive_conf_interval':positive_conf_interval,
#             'positive_samples':len(positive_indecies),
#             'negative_samples':len(negative_indecies)
#         }
#         cluster_to_cluster[compare_cluster] = d
        

In [ ]:
# # activity_regularizer=tf.keras.regularizers.L1L2(l1=0.000001,l2=0.000001)
# def create_body(image_shape):
#     inputs = tf.keras.layers.Input(shape=image_shape)
    
#     def resid_block(layer_inputs, n_filters, kernel_size, strides=1):
#         x = tf.keras.layers.Conv2D(n_filters, kernel_size, padding="same", strides=strides)(layer_inputs)
#         x = tf.keras.layers.BatchNormalization()(x)
#         x = tf.keras.layers.ReLU()(x)
#         x = tf.keras.layers.Conv2D(n_filters, kernel_size, padding="same")(x)
#         x = tf.keras.layers.BatchNormalization()(x)
#         if(strides != 1 or n_filters != layer_inputs.shape[-1]):
#             layer_inputs = tf.keras.layers.Conv2D(n_filters, (1,1), padding="same", strides=strides)(layer_inputs)
#             layer_inputs = tf.keras.layers.BatchNormalization()(layer_inputs)
#         x = x + layer_inputs
#         x = tf.keras.layers.ReLU()(x)
#         return x
    
#     x = resid_block(inputs, config["FIRST_FILTERS"], config["FIRST_KERNEL_SIZE"], strides=1)
#     for _ in range(config["CONV_LAYERS"]):
#         x = resid_block(x, config["N_FILTERS"], config["KERNEL_SIZE"])
    
#     x = tf.keras.layers.GlobalAveragePooling2D()(x)
# #     x = tf.keras.layers.Dense(config["DENSE_SIZE"])(x)
# #     x = tf.keras.layers.BatchNormalization()(x)
# #     x = tf.keras.layers.ReLU()(x)
#     x = tf.keras.layers.Dropout(config["DROPOUT"])(x)
#     outputs = tf.keras.layers.Dense(config["EMBEDDING_SIZE"])(x)
#     return tf.keras.models.Model(inputs=inputs, outputs=outputs, name="body")

# body = create_body(X_train.shape[1:])
# body.summary()